## Code Scrape DF ##

In [113]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import geocoder # import geocoder
import io
import requests
import numpy as np
from sklearn.cluster import KMeans

In [114]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [115]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")

In [116]:
# use the 'find' function to bring back only the 'wikitable sortable' table
right_table=soup.find('table', class_='wikitable sortable')

In [117]:
# create empty lists to store the 'postal code', 'borough' and 'neighbourhood' values for each row of the table
A=[]
B=[]
C=[]

In [118]:
# loop through rows to extraxt 'td' tags

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [119]:
# remove '\n' from all list items
A = [x[:-1] for x in A]
B = [x[:-1] for x in B]
C = [x[:-1] for x in C]

In [120]:
# create dataframe and add lists A-C as columns
df = pd.DataFrame(A,columns=['PostalCode'])
df['Borough'] = B
df['Neighbourhood'] = C

In [121]:
# drop rows if Borough = 'not assigned'
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

In [122]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Latitude & Longitude ##

In [123]:
url = "http://cocl.us/Geospatial_data"
s = requests.get(url).content
ds = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [96]:
ds.rename(columns={"Postal Code":"PostalCode"},inplace=True)
ds.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [97]:

mergedDF = pd.merge(df,ds, on='PostalCode')


In [98]:
mergedDF.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Clustering##

In [99]:
# define foursquarw credentials and version
CLIENT_ID = '05N1XEZQNIDGVQ2REY5LPG4R5QJL22VOJGGTQ5L4RK1EMJFU' # your Foursquare ID
CLIENT_SECRET = 'SEGUNZBYC1ABUB2OYNU1VAKVYU44R03BY3QIQSIUWS5XGQ4B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 05N1XEZQNIDGVQ2REY5LPG4R5QJL22VOJGGTQ5L4RK1EMJFU
CLIENT_SECRET:SEGUNZBYC1ABUB2OYNU1VAKVYU44R03BY3QIQSIUWS5XGQ4B


In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
toronto_venues = getNearbyVenues(names=mergedDF['PostalCode'],
                                   latitudes=mergedDF['Latitude'],
                                   longitudes=mergedDF['Longitude']
                                  )

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [102]:
toronto_venues.shape

(2162, 7)

In [103]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [104]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B,1,1,1,1,1,1
M1C,3,3,3,3,3,3
M1E,8,8,8,8,8,8
M1G,3,3,3,3,3,3
M1H,9,9,9,9,9,9
...,...,...,...,...,...,...
M9N,2,2,2,2,2,2
M9P,7,7,7,7,7,7
M9R,4,4,4,4,4,4


In [105]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [107]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                             venue  freq
0             Fast Food Restaurant   1.0
1                    Metro Station   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----M1C----
                             venue  freq
0                     Home Service  0.33
1                              Bar  0.33
2                      Golf Course  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----M1E----
                 venue  freq
0   Mexican Restaurant  0.12
1       Medical Center  0.12
2       Breakfast Spot  0.12
3           Restaurant  0.12
4  Rental Car Location  0.12


----M1G----
                             venue  freq
0                      Coffee Shop  0.67
1                Korean Restaurant  0.33
2                      Yoga Studio  0.00
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----M1H----
                 ven

4  Mexican Restaurant  0.00


----M4V----
                 venue  freq
0                  Pub  0.12
1          Coffee Shop  0.12
2   Light Rail Station  0.06
3  Fried Chicken Joint  0.06
4           Sports Bar  0.06


----M4W----
                             venue  freq
0                             Park  0.50
1                            Trail  0.25
2                       Playground  0.25
3  Molecular Gastronomy Restaurant  0.00
4       Modern European Restaurant  0.00


----M4X----
         venue  freq
0  Pizza Place  0.06
1  Coffee Shop  0.06
2   Restaurant  0.04
3         Café  0.04
4         Park  0.04


----M4Y----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.05
3              Gay Bar  0.04
4           Restaurant  0.04


----M5A----
            venue  freq
0     Coffee Shop  0.17
1             Pub  0.06
2            Park  0.06
3          Bakery  0.06
4  Breakfast Spot  0.04


----M5B----
                 venue  

                             venue  freq
0                      Pizza Place   1.0
1                      Yoga Studio   0.0
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----M9M----
                             venue  freq
0                      Fabric Shop   0.5
1                   Baseball Field   0.5
2                      Yoga Studio   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----M9N----
                             venue  freq
0                             Park   0.5
1                Convenience Store   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


----M9P----
                venue  freq
0         Pizza Place  0.29
1      Discount Store  0.14
2  Chinese Restaurant  0.14
3         Coffee Shop  0.14
4        Intersection  0.14


----M9R----
                venue  freq
0         

In [108]:
toronto_grouped.shape

(100, 272)

In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [110]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M1C,Golf Course,Bar,Home Service,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
2,M1E,Electronics Store,Medical Center,Restaurant,Mexican Restaurant,Bank,Intersection,Rental Car Location,Breakfast Spot,Doner Restaurant,Donut Shop
3,M1G,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
4,M1H,Caribbean Restaurant,Gas Station,Bakery,Bank,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Lounge,Dumpling Restaurant


In [111]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 2, 1, 1, 1, 1], dtype=int32)

In [112]:
# add clustering labels
neighborhoods_venues_sorted = neighborhoods_venues_sorted.reset_index(level=0, drop=True).reset_index()
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = mergedDF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

KeyError: 'Neighborhood'